# ИД23-1 Маслов АН
## Измененные примеры из лекции по SQLlite3

### Создание БД (изменены таблицы)

In [11]:
import sqlite3

# устанавливаем соединение с БД
con = sqlite3.connect('maslov.db')
# создаем объект-курсор
cur = con.cursor()
#создаем простую БД
sql = """
CREATE TABLE USER
(
USER_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
EMAIL VARCHAR(200) NOT NULL UNIQUE,
PASSWORD VARCHAR(200) NOT NULL,
ROLE VARCHAR(50) NOT NULL
);
CREATE TABLE AUTHOR
(
AUTHOR_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
NAME VARCHAR(200),
CONTACT VARCHAR(200),
USER_ID INTEGER UNIQUE,
FOREIGN KEY (USER_ID) REFERENCES USER (USER_ID)
);
CREATE TABLE BOOK
(
BOOK_ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
AUTHOR_ID INTEGER NOT NULL,
TITLE VARCHAR(200) NOT NULL UNIQUE,
YEAR INTEGER,
FOREIGN KEY (AUTHOR_ID) REFERENCES AUTHOR (AUTHOR_ID)
);
"""

try: # Обрабатываем исключения
    cur.executescript(sql) # Выполняем SQL-запросы
except sqlite3.DatabaseError as err:
    print("Ошибка:", err)
else:
    print ("Запрос успешно выполнен")
cur.close()
con.close()



Запрос успешно выполнен


### Добавление записей в таблицу (изменены вносимые данные и показаны пара способов внесения)

In [12]:
con = sqlite3.connect('maslov.db')
cur = con.cursor()

#SQL-запрос для вставки значений в таблицу USER
sql_1 = '''
INSERT INTO USER (EMAIL, PASSWORD, ROLE)
VALUES (?, ?, ?);'''
v1 = ('admin@inbox.com', 'password123', 'admin')
v2 = ('alex@mail.ru', '123456', 'user')
v3 = ('ivan@yandex.ru', 'qwerty', 'author')

val_list = [v1, v2, v3]


#SQL-запрос для вставки значений в таблицу AUTHOR
sql_2 = ''' 
INSERT INTO AUTHOR (NAME, CONTACT, USER_ID)
VALUES (?, ?, ?);'''
v4 = ('Александр', '+7(999)123-45-67', None)
v5 = ('Иван', '+7(999)234-56-78', 3)

val_list_2 = [v4, v5]

#SQL-запрос для вставки значений в таблицу BOOK
sql_3 = '''
INSERT INTO BOOK (AUTHOR_ID, TITLE, YEAR)
VALUES (?, ?, ?);'''
v6 = (1, 'Книга', 2009)

try:
    cur.executemany(sql_1, val_list) # Вставляем несколько записей
    cur.executemany(sql_2, val_list_2)
    cur.execute(sql_3, v6) # Вставляем одну запись
except sqlite3.DatabaseError as err:
    print("Ошибка:", err)
else:
    print ("Запрос успешно выполнен")
con.commit() # Завершаем транзакцию
cur.close() # Закрываем объект-курсор
con.close() # Закрываем соединение


Запрос успешно выполнен


### Обработка результата запроса (вывод показан через JOIN)

In [16]:
con = sqlite3.connect("maslov.db")

print("Таблица AUTHOR:")
cur = con.cursor()
cur.execute('SELECT * FROM AUTHOR') 
for v in cur: #вывод результата запроса
    print(v)

print("Таблица BOOK и AUTHOR через JOIN:")
cur.execute('SELECT TITLE, NAME, YEAR FROM BOOK JOIN AUTHOR ON BOOK.AUTHOR_ID = AUTHOR.AUTHOR_ID')
for v in cur:
    print(v)

cur.close()
con.close()

Таблица AUTHOR:
(1, 'Александр', '+7(999)123-45-67', None)
(2, 'Иван', '+7(999)234-56-78', 3)
Таблица BOOK и AUTHOR через JOIN:
('Книга', 'Александр', 2009)


### Управление транзакциями (показана работа rollback)

In [14]:
con = sqlite3.connect("maslov.db")
cur = con.cursor()
# SQL выражение
sql = '''
INSERT INTO USER (EMAIL, PASSWORD, ROLE)
VALUES (?, ?, ?)'''
val = ('New value', 'test', 'test')
try:
    cur.execute(sql, val)
    con.rollback() # Отмена транзакции
    cur.execute('SELECT * FROM USER WHERE EMAIL = ?', (val[0],)) #проверяем вставку
    print('Результат вставки', cur.fetchall())
except sqlite3.DatabaseError as err:
    print("Ошибка:", err)
else:
    print ("Запрос успешно выполнен")
con.commit() # Завершаем транзакцию
cur.close() # Закрываем объект-курсор
con.close() # Закрываем соединение

Результат вставки []
Запрос успешно выполнен


### Обработка исключений (показана проверка исключения через другое исключение)

In [19]:
con = sqlite3.connect("maslov.db")
cur = con.cursor()
# SQL выражение
sql = '''
INSERT INTO USER (EMAIL)
VALUES (?)'''
val = ('New value 3',)
try:
    with con:
        con.execute(sql, val)
except sqlite3.IntegrityError as err:
    print("Ошибка:", err)
else:
    print ("Запрос успешно выполнен")
cur.close() # Закрываем объект-курсор
con.close() # Закрываем соединение

Ошибка: NOT NULL constraint failed: USER.PASSWORD
